In [1]:
import numpy as np
import pandas as pd
import re
import time
from tqdm.notebook import tqdm

# 지도 + 시각화
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc #한글지원
import platform
import seaborn as sns
plt.rc("font",family='Malgun Gothic') #한글_글꼴
plt.rcParams['axes.unicode_minus'] = False #음수표시

import folium

# 거리계산
from haversine import haversine #패키지 설치필요

In [11]:
### 지도시각화
class Mapping:

    def __init__(self, s_lat=37.758206, s_lng=128.806648):
        file_path = './data/'
        self.return_map = folium.Map(location=[s_lat, s_lng],
                                     tiles='openstreetmap',
                                     zoom_start=50)
        self.sleep = pd.read_excel(file_path + '졸음쉼터+편의시설.xlsx')
        self.rest = pd.read_csv(file_path + '휴게소정보_20210809.csv',
                                encoding='euc-kr')
        self.danger = pd.read_csv(file_path + 'danger.csv')
        self.tunnel = pd.read_excel(file_path + '터널_위치_수정.xlsx')
        self.crush_bound_cnt = pd.read_excel(file_path + '졸음쉼터_주변사고.xlsx')
        self.rest = self.rest.iloc[:, :-4]
        for val in self.rest.columns[12:]:
            self.rest = self.rest.rename(columns={val: val[:-2]})  #변수명 변경
        self.icon_lis = list('🅿🪛⛽⛽🔌🚌🛏🚻💊🚼🍴😋')
        self.icon_lis2 = list('🅿🪑⛱💪🍴⛟🍹')

    ### 휴게소 표시
    def rest_map(self):
        for i in range(0, len(self.rest)):
            lat = self.rest.loc[i, '위도']  #위도
            lng = self.rest.loc[i, '경도']  #경도

            str_pop = ''
            for j, icon in zip(range(11, len(self.rest.columns)),
                               self.icon_lis):
                if j == len(self.rest.columns) - 1:
                    str_pop += '<b>{}</b> | {}'.format(
                        self.rest.columns[j] + icon, self.rest.iloc[i, j])
                else:
                    str_pop += '<b>{}</b> | {} <br/>'.format(
                        self.rest.columns[j] + icon, self.rest.iloc[i, j])

            popup = folium.Popup(str_pop, max_width=200)

            folium.Marker(location=[lat, lng],
                          tooltip=self.rest.loc[i, '휴게소명'],
                          popup=popup,
                          icon=folium.Icon(color='blue',
                                           icon='utensils',
                                           prefix='fa')).add_to(
                                               self.return_map)  #마커 표시
        return self.return_map

    ### 졸음쉼터 표시
    def sleep_map(self):
        _, fc_50, fc_75 = np.percentile(self.crush_bound_cnt['cnt'], [25, 50, 75])
        for i in range(0, len(self.sleep)):
            lat = self.sleep.loc[i, '위도']  #위도
            lng = self.sleep.loc[i, '경도']  #경도

            str_pop = ''
            for j, icon in zip(range(4, len(self.sleep.columns)),
                               self.icon_lis2):
                if j == len(self.sleep.columns) - 1:
                    str_pop += '<b>{}</b> | {}'.format(
                        self.sleep.columns[j] + icon, self.sleep.iloc[i, j])
                else:
                    str_pop += '<b>{}</b> | {} <br/>'.format(
                        self.sleep.columns[j] + icon, self.sleep.iloc[i, j])

            popup = folium.Popup(str_pop, max_width=200)
            
            if self.sleep.loc[i,'졸음쉼터명'] in self.crush_bound_cnt['졸음쉼터명'].unique():
                temp = self.crush_bound_cnt[self.crush_bound_cnt['졸음쉼터명'] == self.sleep.loc[i,'졸음쉼터명']]['cnt'].values[0]
                if temp>=fc_75:
                    f_c = 'red' #주변 사고 발생률이 상위 25%인 졸음쉼터는 빨간색 마커
                elif temp>=fc_50:
                    f_c = 'orange' #주변 사고 발생률이 상위 50%인 졸음쉼터는 오렌지색 마커
                else:
                     f_c = 'green'
            else:
                f_c = 'cadetblue'

            folium.Marker(location=[lat, lng],
                          tooltip=self.sleep.loc[i, '졸음쉼터명'],
                          popup=popup,
                          icon=folium.Icon(color=f_c,
                                           icon='bed',
                                           prefix='fa')).add_to(
                                               self.return_map)
        return self.return_map

    ### 사고다발지역 표시
    def crush_circle(self):
        for i in range(0, len(self.danger)):
            if self.danger['danger_rank'][i] == 3:
                f_c = 'orange'
            else:
                f_c = 'red'

            folium.Circle(
                location=[self.danger.iloc[i, -2], self.danger.iloc[i, -1]],
                radius=100,
                color=f_c,
                fill_color=f_c).add_to(self.return_map)
        return self.return_map

    ### 터널위치 표시
    def tunnel_poly(self):
        for i in self.tunnel.index:
            folium.PolyLine(
                locations=[[self.tunnel.iloc[i, 4], self.tunnel.iloc[i, 5]],
                           [self.tunnel.iloc[i, 6], self.tunnel.iloc[i, 7]]],
                color='gray',
                tooltip=self.tunnel.iloc[i, 0],
                width=3).add_to(self.return_map)
        return self.return_map

# class test

In [14]:
a = Mapping()
a.sleep_map()

In [15]:
a = Mapping()
a.rest_map()